<a href="https://colab.research.google.com/github/neuralsrg/SupervisedLearning/blob/main/Tensorflow/%5Bhand_signs%5Dtf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating the same Neural Net (as in tf_manual_nn) using tf functions

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Dataset

In [3]:
import h5py

train_dataset = h5py.File('/content/drive/MyDrive/HandSigns/train_signs.h5', "r")
test_dataset = h5py.File('/content/drive/MyDrive/HandSigns/test_signs.h5', "r")

In [94]:
X_train_old = np.array(train_dataset['train_set_x']).reshape(-1, 64 * 64 * 3) / 255.
X_test_old = np.array(test_dataset['test_set_x']).reshape(-1, 64 * 64 * 3) / 255.
y_train_old = np.array(train_dataset['train_set_y'])
y_test_old = np.array(test_dataset['test_set_y'])

X_train_old = X_train_old.reshape(-1, 1, 64*64*3)
X_test_old = X_test_old.reshape(-1, 1, 64*64*3)

X_train_old = X_train_old[0].reshape(1, 1, -1)
y_train_old = np.array([y_train_old[0]])

X_train_old = tf.cast(tf.constant(X_train_old), dtype=tf.float32)
y_train_old = tf.constant(y_train_old)

In [95]:
X_train_old.shape

TensorShape([1, 1, 12288])

## Model

In [96]:
tf.random.set_seed(32)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(25,
                                activation='relu',
                                kernel_initializer='glorot_uniform'))
model.add(tf.keras.layers.Dense(12,
                                activation='relu',
                                kernel_initializer='glorot_uniform'))
model.add(tf.keras.layers.Dense(6,
                                activation='softmax',
                                kernel_initializer='glorot_uniform'))

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [97]:
model(X_train_old)#.reshape(-1, 1, 64*64*3))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (1, 12288)                0         
                                                                 
 dense_15 (Dense)            (1, 25)                   307225    
                                                                 
 dense_16 (Dense)            (1, 12)                   312       
                                                                 
 dense_17 (Dense)            (1, 6)                    78        
                                                                 
Total params: 307,615
Trainable params: 307,615
Non-trainable params: 0
_________________________________________________________________


In [98]:
# parameters

W1, b1 = model.layers[1].get_weights()
tf_W1_old = np.copy(W1)
W1 = tf.Variable(W1)
b1 = tf.Variable(b1.reshape(1, -1))

W2, b2 = model.layers[2].get_weights()
W2 = tf.Variable(W2)
b2 = tf.Variable(b2.reshape(1, -1))

W3, b3 = model.layers[3].get_weights()
W3 = tf.Variable(W3)
b3 = tf.Variable(b3.reshape(1, -1))

parameters = {
    'W1' : W1,
    'b1' : b1,
    'W2' : W2,
    'b2' : b2,
    'W3' : W3,
    'b3' : b3,
}

## Training

In [100]:
# lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 0.99**(epoch))

history = model.fit(
    x=X_train_old,#.reshape(-1, 1, 64*64*3),
    y=y_train_old,
    batch_size=1,
    epochs=1,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - 0s 286ms/step - loss: 1.8720 - accuracy: 0.0000e+00


In [101]:
W1, b1 = model.layers[1].get_weights()
tf_W1_new = np.copy(W1)

## Manual

In [93]:
train_dataset = h5py.File('/content/drive/MyDrive/HandSigns/train_signs.h5', "r")
test_dataset = h5py.File('/content/drive/MyDrive/HandSigns/test_signs.h5', "r")

x_train = tf.data.Dataset.from_tensor_slices(train_dataset['train_set_x'])
y_train = tf.data.Dataset.from_tensor_slices(train_dataset['train_set_y'])

x_test = tf.data.Dataset.from_tensor_slices(test_dataset['test_set_x'])
y_test = tf.data.Dataset.from_tensor_slices(test_dataset['test_set_y'])

def normalize(image):
  image = tf.cast(image, tf.float32) / 255.0
  image = tf.reshape(image, [-1, ])
  
  return image

norm_train = x_train.map(normalize)
norm_test = x_test.map(normalize)

def one_hot(label, depth=6):
  return tf.reshape(tf.one_hot(label, depth, axis=0), [1, depth])

norm_y_train = y_train.map(one_hot)
norm_y_test = y_test.map(one_hot)

def forward(X, parameters):

  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']
  W3 = parameters['W3']
  b3 = parameters['b3']

  #print(X.shape, W1.shape, b1.shape)
  Z1 = tf.math.add(tf.linalg.matmul(X, W1), b1)
  A1 = tf.keras.activations.relu(Z1)

  #print(A1.shape, W2.shape, b2.shape)
  Z2 = tf.math.add(tf.linalg.matmul(A1, W2), b2)
  A2 = tf.keras.activations.relu(Z2)

  Z3 = tf.math.add(tf.linalg.matmul(A2, W3), b3)
  A3 = tf.keras.activations.softmax(Z3)
  
  return A3

def cost(preds, labels):

  # tf.keras.categorical_crossentropy expects both preds and lables of shape 
  # (num_samples, num_classes)
  cost = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(labels, preds))

  return cost

In [85]:
def modeling(X_train, Y_train, lr=0.0001):
  
  # optimizer - Adam
  optimizer = tf.keras.optimizers.SGD(lr)

  with tf.GradientTape() as tape:
    A3 = forward(X_train, parameters)
    minibatch_cost = cost(A3, Y_train)

  print(f'Minibatch cost was: {minibatch_cost}')

  trainable = parameters.values()

  # back propagation
  grads = tape.gradient(minibatch_cost, trainable)
  optimizer.apply_gradients(zip(grads, trainable))

  return parameters

In [99]:
man_W1_old = np.copy(parameters['W1'])
parameters = modeling(tf.reshape(X_train_old, shape=(1, -1)), one_hot(y_train_old))
man_W1_new = np.copy(parameters['W1'])

Minibatch cost was: 1.8720366954803467


## Comparing 

In [102]:
tf_W1_old

array([[ 0.00199581, -0.01617085, -0.00210474, ...,  0.00400415,
        -0.00541233,  0.01954625],
       [-0.00774827, -0.01985236,  0.00654225, ...,  0.01703717,
         0.00904857, -0.02189378],
       [-0.01986675, -0.00848466,  0.00357587, ..., -0.01890617,
        -0.0158468 ,  0.0142203 ],
       ...,
       [-0.01084797,  0.00037441, -0.02203674, ..., -0.00582635,
        -0.01651511,  0.00842013],
       [-0.01125425, -0.00640712,  0.00845467, ..., -0.0018795 ,
        -0.01937845, -0.00609114],
       [ 0.01956955, -0.00456122,  0.00275667, ..., -0.00196295,
         0.0200613 ,  0.00113762]], dtype=float32)

In [103]:
man_W1_old

array([[ 0.00199581, -0.01617085, -0.00210474, ...,  0.00400415,
        -0.00541233,  0.01954625],
       [-0.00774827, -0.01985236,  0.00654225, ...,  0.01703717,
         0.00904857, -0.02189378],
       [-0.01986675, -0.00848466,  0.00357587, ..., -0.01890617,
        -0.0158468 ,  0.0142203 ],
       ...,
       [-0.01084797,  0.00037441, -0.02203674, ..., -0.00582635,
        -0.01651511,  0.00842013],
       [-0.01125425, -0.00640712,  0.00845467, ..., -0.0018795 ,
        -0.01937845, -0.00609114],
       [ 0.01956955, -0.00456122,  0.00275667, ..., -0.00196295,
         0.0200613 ,  0.00113762]], dtype=float32)

In [104]:
tf_W1_new

array([[ 0.00200175, -0.01617463, -0.00210474, ...,  0.00400415,
        -0.00541233,  0.01954631],
       [-0.00774251, -0.01985603,  0.00654225, ...,  0.01703717,
         0.00904857, -0.02189372],
       [-0.01986115, -0.00848823,  0.00357587, ..., -0.01890617,
        -0.0158468 ,  0.01422036],
       ...,
       [-0.01084253,  0.00037094, -0.02203674, ..., -0.00582635,
        -0.01651511,  0.00842018],
       [-0.01124877, -0.0064106 ,  0.00845467, ..., -0.0018795 ,
        -0.01937845, -0.00609108],
       [ 0.01957503, -0.0045647 ,  0.00275667, ..., -0.00196295,
         0.0200613 ,  0.00113768]], dtype=float32)

In [105]:
man_W1_new

array([[ 0.00200175, -0.01617463, -0.00210474, ...,  0.00400415,
        -0.00541233,  0.01954631],
       [-0.00774251, -0.01985603,  0.00654225, ...,  0.01703717,
         0.00904857, -0.02189372],
       [-0.01986115, -0.00848823,  0.00357587, ..., -0.01890617,
        -0.0158468 ,  0.01422036],
       ...,
       [-0.01084253,  0.00037094, -0.02203674, ..., -0.00582635,
        -0.01651511,  0.00842018],
       [-0.01124877, -0.0064106 ,  0.00845467, ..., -0.0018795 ,
        -0.01937845, -0.00609108],
       [ 0.01957503, -0.0045647 ,  0.00275667, ..., -0.00196295,
         0.0200613 ,  0.00113768]], dtype=float32)

In [162]:
(tf_W1_old - tf_W1_new) / 0.0001

array([[-0.99997735,  0.9999424 ,  0.        , ...,  0.        ,
         0.        , -0.9947829 ],
       [-0.999975  ,  0.9999424 ,  0.        , ...,  0.        ,
         0.        , -0.99461526],
       [-0.9999797 ,  0.9999424 ,  0.        , ...,  0.        ,
         0.        , -0.99446625],
       ...,
       [-0.9999704 ,  0.99993867,  0.        , ...,  0.        ,
         0.        , -0.99430794],
       [-0.9999704 ,  0.9999378 ,  0.        , ...,  0.        ,
         0.        , -0.9943359 ],
       [-0.9999797 ,  0.9999378 ,  0.        , ...,  0.        ,
         0.        , -0.9943359 ]], dtype=float32)

In [163]:
(man_W1_old - man_W1_new) / 0.0001

array([[ 0.9996805 ,  1.0000169 , -0.99994946, ..., -0.99999833,
        -1.0000216 ,  0.99999833],
       [ 0.99967235,  1.0000169 , -0.99994713, ..., -0.99999833,
        -1.0000169 ,  0.99999833],
       [ 0.99966305,  1.0000169 , -0.99994475, ..., -0.99999833,
        -1.0000169 ,  0.99999833],
       ...,
       [ 0.9996444 ,  1.0000123 , -0.9999424 , ..., -0.99999833,
        -1.0000169 ,  0.99999833],
       [ 0.99965376,  1.0000123 , -0.9999424 , ..., -0.999996  ,
        -1.0000169 ,  0.99999833],
       [ 0.9996444 ,  1.0000123 , -0.9999424 , ..., -0.999996  ,
        -1.0000169 ,  0.99999714]], dtype=float32)

## Saving model

The SavedModel format (default)

In [ ]:
model.save('/content/drive/MyDrive/HandSigns/SavedModel_format')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/HandSigns/SavedModel_format/assets


In [ ]:
!ls /content/drive/MyDrive/HandSigns/SavedModel_format

assets	keras_metadata.pb  saved_model.pb  variables


The **HDF5** format

In [ ]:
model.save('/content/drive/MyDrive/HandSigns/saved_model.h5')

In [ ]:
!ls /content/drive/MyDrive/HandSigns/

SavedModel_format  saved_model.h5  test_signs.h5  train_signs.h5


## Loading model

In [ ]:
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/HandSigns/SavedModel_format")
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 25)                307225    
                                                                 
 dense_4 (Dense)             (None, 12)                312       
                                                                 
 dense_5 (Dense)             (None, 6)                 78        
                                                                 
Total params: 307,615
Trainable params: 307,615
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/HandSigns/saved_model.h5")
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 25)                307225    
                                                                 
 dense_4 (Dense)             (None, 12)                312       
                                                                 
 dense_5 (Dense)             (None, 6)                 78        
                                                                 
Total params: 307,615
Trainable params: 307,615
Non-trainable params: 0
_________________________________________________________________
